## Load model

In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import numpy as np

def extract_attack_kwargs(train=False, eval=True):
    args = None # delete this
    l = []

    def get_ord(ordstr):
        if ordstr == 'inf':
            return np.inf
        else:
            return int(ordstr)
    if train:
        train_kwargs = {
            'out_dir': args.out_path,
            'adv_train': int(args.adv_train),
            'constraint': '2' if getattr(args, "metrics_constraint", None) is None else args.metrics_constraint,
            'eps': args.eps_train,
            'attack_lr': args.lr_train,
            'attack_steps': args.iters_train,
            'ord': get_ord(args.ord_train),
            'constraint': args.ord_train,
            'epochs': args.epochs,
            'step_lr': args.epochs//args.lr_steps,
            'lr': args.lr_opt,
            'random_start': True
        }
        l.append(train_kwargs)
    if eval:
        attack_kwargs = {
            "eps": 0.0356, # args.eps_eval,
            "nb_iter": 20, # args.iters_eval,
            "eps_iter": 0.01, # args.lr_eval,
            'ord': get_ord('inf')
        }
        l.append(attack_kwargs)
    return tuple(l) if len(l) > 1 else l[0]

# Define the image transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [2]:
# Load the pretrained ResNet18 model
net = models.resnet18(pretrained=True)
net.eval()

# Load an image
img_path = "retriever-smiling-cropped.jpg"
input_image = Image.open(img_path)
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Create a mini-batch as expected by the model

# Move the input to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_batch = input_batch.to(device)
net.to(device)

output = net(input_batch)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top_prob, top_catid = torch.topk(probabilities, 5)

# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt") as f:
    labels = [line.strip() for line in f.readlines()]

for i in range(top_prob.size(0)):
    print(labels[top_catid[i]], top_prob[i].item())

golden retriever 0.9293431639671326
Labrador retriever 0.021033722907304764
clumber 0.013683471828699112
Great Pyrenees 0.009099332615733147
otterhound 0.004993060603737831


C:\Users\u_imagenes\.conda\envs\sparsity\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\u_imagenes\.conda\envs\sparsity\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
attack_kwargs = extract_attack_kwargs()

In [4]:
attack_kwargs

{'eps': 0.0356, 'nb_iter': 20, 'eps_iter': 0.01, 'ord': inf}